In [1]:
!pip install transformers datasets torch
!pip install transformers[torch]

Looking in indexes: https://pypi.python.org/simple/
  Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached torch-2.2.2-cp38-cp38-manylinux1_x86_64.whl (755.5 MB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached safetensors-0.4.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached tokenizers-0.15.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Using cached filelock-3.13.4-py3-none-any.whl (11 kB)
  Using cached regex-2023.12.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (777 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached pyarrow-15.0.2-cp38-cp38-manylinux_2_28_x86_64.whl (38.4 MB)
  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached xxh

In [2]:
#!pip uninstall accelerate
!pip install accelerate==0.28

Looking in indexes: https://pypi.python.org/simple/
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.2
    Uninstalling accelerate-0.29.2:
      Successfully uninstalled accelerate-0.29.2

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from datasets import load_dataset

dataset = load_dataset("boolq")

# Tokenized the question and passage
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

def preprocess_function(datadict):
    return tokenizer(datadict["question"], datadict["passage"], truncation=True, padding=True, max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [4]:
# Corrected function to convert boolean labels to integers

def convert_to_tensor(dataset):
    # Correctly handle conversion of boolean 'answer' to integer
    dataset['labels'] = int(dataset['answer'])
    return dataset

# Correctly apply the conversion function to each dataset split
for split in tokenized_datasets.keys():
    tokenized_datasets[split] = tokenized_datasets[split].map(convert_to_tensor)

tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [5]:
# Load pretrained model from HF

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

from transformers import TrainingArguments, Trainer
from accelerate import Accelerator, DataLoaderConfiguration

# dataloader config in accelerator
dataloader_config = DataLoaderConfiguration(split_batches=False, even_batches=True, use_seedable_sampler=True)
accelerator = Accelerator(dataloader_config=dataloader_config)

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=3,
                                  warmup_steps=500,
                                  per_device_train_batch_size = 32,
                                  learning_rate = 2e-5
                                  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [6]:
print(accelerator.device)

cpu


In [7]:
import torch

# Check if CUDA is available
print(torch.cuda.is_available())

False


In [8]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
# Train model
trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,No log,0.656372


In [ ]:
# Save model
model_path = "./finetuned_base_bert_boolq"

# Save the model
model.save_pretrained(model_path)

# Save the tokenizer associated with the model
tokenizer.save_pretrained(model_path)